<a href="https://colab.research.google.com/github/nickname8888/pyspark-prac/blob/main/udfs_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark nltk

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [3]:
spark = SparkSession.builder.appName("Analyzer").getOrCreate()

In [6]:
# using the sentiment analysis dataset

df = spark.read.csv("/content/IMDB Dataset.csv", header=True, inferSchema=True)

In [7]:
df.show(10)

+--------------------+--------------------+
|              review|           sentiment|
+--------------------+--------------------+
|One of the other ...|            positive|
|"A wonderful litt...| not only is it w...|
|"I thought this w...| but spirited you...|
|Basically there's...|            negative|
|"Petter Mattei's ...| power and succes...|
|"Probably my all-...| but that only ma...|
|I sure would like...|            positive|
|This show was an ...|            negative|
|Encouraged by the...|            negative|
|If you like origi...|            positive|
+--------------------+--------------------+
only showing top 10 rows



In [8]:
df.printSchema()

root
 |-- review: string (nullable = true)
 |-- sentiment: string (nullable = true)



+--------------------+
|           sentiment|
+--------------------+
|            positive|
| not only is it w...|
| but spirited you...|
|            negative|
| power and succes...|
| but that only ma...|
|            positive|
|            negative|
|            negative|
|            positive|
|            negative|
|            negative|
| meaning most of ...|
|            negative|
|            positive|
|            negative|
|            positive|
|            negative|
|            positive|
|            negative|
+--------------------+
only showing top 20 rows

